In [51]:
import numpy as np
import pandas as pd
import os
import re

In [121]:
requests_sd = pd.read_excel('отгруженное оборудование ПРЦ.xlsx', sheet_name='Заявки')
wizar_pos = pd.read_excel('отгруженное оборудование ПРЦ.xlsx', sheet_name='POS')
tsos = pd.read_excel('отгруженное оборудование ПРЦ.xlsx', sheet_name='ТСО')
ptrn_wizar_serial_number = 'WP\d{5}Q\d{8}'

'#UN0002##ViVOpay kiosk III контроллер#204H314110#ViVOpay kiosk III антенна#207G196120##2000005453489#1####2000005453502#1##  Произведена установка NFC контроллер 204H314110 антенна 207G196120'

### Получения списка серийников для NFC ридеров

In [164]:
ptrn_tso_antena_sn = '\d{3}[G]?\d{6}'
ptrn_tso_kontrol_sn = '\d{3}[H]?\d{6}'
wizar_pos_istall = requests_sd[requests_sd['Решение / Обходной путь'].str.contains('#UN0002')]
wizar_pos_istall.index = np.arange(0, len(wizar_pos_istall['ID']))
seria_solution = dict(wizar_pos_istall[['ID', 'КЭ Адрес', 'Решение / Обходной путь']])

In [171]:
dict_ids_tso = {
    'ID': [],
    'Решение': [],
    'Адрес': [],
    'Антенна': [],
    'Контроллер': []
}

In [175]:
for i in range(len(seria_solution['Решение / Обходной путь'])):
    result_antena = re.search(ptrn_tso_antena_sn, seria_solution['Решение / Обходной путь'][i])
    result_kontrol = re.search(ptrn_tso_kontrol_sn, seria_solution['Решение / Обходной путь'][i])
    dict_ids_tso['ID'].append(seria_solution['ID'][i])
    dict_ids_tso['Решение'].append(seria_solution['Решение / Обходной путь'][i])
    dict_ids_tso['Адрес'].append(seria_solution['КЭ Адрес'][i])
    try:
        dict_ids_tso['Антенна'].append(result_antena.group(0))
    except AttributeError:
        dict_ids_tso['Антенна'].append(0)
        
    try:
        dict_ids_tso['Контроллер'].append(result_kontrol.group(0))
    except AttributeError:
        dict_ids_tso['Контроллер'].append(0)

pd.DataFrame(dict_ids_tso).to_excel('tso_sns.xlsx')
#for key, val in dict_ids_tso.items():
#    print(f'{key}: {len(val)}')
    


### Получения списка серийников для WizarPOS

In [104]:
wizar_pos_solution_df = wizar_pos_unistall[['ID', 'Решение / Обходной путь', 'КЭ Адрес', 'КЭ Код ТБ']]

In [118]:
wizar_pos_solution = list(wizar_pos_unistall['Решение / Обходной путь'])
wizar_pos_istall_list = []
wizar_pos_istall_incorrect_sn_list = []

for val in wizar_pos_solution:
    result = re.search(ptrn_wizar_serial_number, val)
    try:
        wizar_pos_istall_list.append(result.group(0))
    except AttributeError:
        wizar_pos_istall_list.append(val)

len(wizar_pos_istall_list)

wizar_pos_solution_df['SN'] = wizar_pos_istall_list
wizar_pos_solution_df.index = np.arange(1, len(wizar_pos_solution_df['SN']) + 1)
#pd.DataFrame(wizar_pos_solution_df['SN'].value_counts()).to_excel('Counts.xlsx')
len(wizar_pos_solution_df['SN'].unique())

C:\Users\kuzinmy\AppData\Local\Temp/ipykernel_15184/1339458861.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wizar_pos_solution_df['SN'] = wizar_pos_istall_list


460

In [80]:
wizar_pos_istall_df = pd.DataFrame(wizar_pos_istall_list, columns=['SN'])
wizar_pos_istall_df['SN'].unique()

456